## Finding Lane Lines

## Import Packages

In [15]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions and Global Variables 

Appart from the functions provided three new functions are added here. 
1. reset_globals()
2. hls()
3. lane_pass_filter()

In [16]:
import math
#Global variables
"These variables are used to store the x and y co-ordinates of the last 5 frames." 
glbx =[]
gyb =0
gltx =[]
gyt =0
grbx =[]
grtx =[]


def reset_globals():
    "Resets the global variables"
    global glbx, gyb, gltx, gyt, grbx, grtx
    glbx =[]
    gyb =0
    gltx =[]
    gyt =0
    grbx =[]
    grtx =[]
    
def grayscale(img):
    "Applies the Grayscale transform"
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    
def hls(img):
    "convert the image to HSL (Hue, Saturation, Lightness) space"
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

def lane_pass_filter(img):
    "filters out everything(mostly) that is not white or yellow "
    hls_image = hls(img)
    l_thresh = np.uint8([  0, 200,   0])
    u_thresh = np.uint8([255, 255, 255])
    white_pass = cv2.inRange(hls_image, l_thresh, u_thresh)
    l_thresh = np.uint8([ 12,   0, 100])
    u_thresh = np.uint8([ 45, 255, 255])
    yellow_pass = cv2.inRange(hls_image, l_thresh, u_thresh)
    yellow_white_pass = cv2.bitwise_or(white_pass, yellow_pass)
    return cv2.bitwise_and(img, img, mask = yellow_white_pass)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines,end_point, color=[255, 0, 0], thickness=10):
    "Creates the best approximation of a left and right lane line"
    global glbx, gyb, gltx, gyt, grbx, grtx
    #left and right lane line slope
    left=[]
    right=[]
    #left and right x and y coordinates 
    left_x =[]
    left_y =[]
    right_x =[]
    right_y =[]
    imshape = img.shape
    # only use this if there are lines to begin with else use running average from last 5 frames
    if lines !=None:
            for line in lines:
                for x1,y1,x2,y2 in line:
                    slope = ((y2-y1)/(x2-x1))
                    if (abs(slope) >= .5 and abs(slope)<=2):
                        if (slope>0 and x1>(imshape[1]/2) and  x1>(imshape[1]/2) ):
                            left_x.append(x1)
                            left_x.append(x2)
                            left_y.append(y1)
                            left_y.append(y2)
                            left.append(slope)
                        elif (slope<0  and x1<(imshape[1]/2) and  x1<(imshape[1]/2) ):
                            right_x.append(x1)
                            right_x.append(x2)
                            right_y.append(y1)
                            right_y.append(y2)                
                            right.append(slope)
                            
    if len(left)!=0 and len(right)!=0:
        # find the average slope for left and right lane
        left_slope = float(sum(left)) / len(left)
        right_slope = float(sum(right)) / len(right)
        # find average x and y position for each lane
        l_x = int(sum(left_x) / len(left_x))
        l_y = int(sum(left_y) / len(left_y))
        r_x = sum(right_x) / len(right_x)
        r_y = sum(right_y) / len(right_y)
        # The endpoints for the extrapolated line 
        yb =imshape[0]
        yt =int(end_point)
        # The 'c' in y=mx+c 
        lc = l_y-(left_slope*l_x)
        # Find the top and bottom points using the calculated 'c'.
        lbx= int((yb-lc)/left_slope)
        ltx= int((yt-lc)/left_slope)
        # The 'c' for right lane
        rc = r_y-(right_slope*r_x)
        rbx= int((yb-rc)/right_slope)
        rtx= int((yt-rc)/right_slope)
        # maintain the values for last 5 frames in global variables
        if len(glbx)> 6:
            glbx.pop(0)
            gltx.pop(0)
            grbx.pop(0)
            grtx.pop(0)
        glbx.append(lbx)
        gyb =yb
        gltx.append(ltx)
        gyt =yt
        grbx.append(rbx)
        grtx.append(rtx)
        # use the average value from last 5 frames to counter hyper-sensitivity of lane line
        lbx =int(int(sum(glbx) / len(glbx)))
        ltx =int(int(sum(gltx) / len(gltx)))
        rbx =int(int(sum(grbx) / len(grbx)))
        rtx =int(int(sum(grtx) / len(grtx)))         
            
    else:
        # use stored average if the lane in current frame is not recognized
        yb =gyb
        yt =gyt
        lbx =int(int(sum(glbx) / len(glbx)))
        ltx =int(int(sum(gltx) / len(gltx)))
        rbx =int(int(sum(grbx) / len(grbx)))
        rtx =int(int(sum(grtx) / len(grtx))) 
        
    
    cv2.line(img, (lbx,yb),(ltx,yt), color, thickness)
    cv2.line(img, (rbx,yb),(rtx,yt), color, thickness)
 
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap,end_point):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines,end_point)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


## Build a Lane Finding Pipeline



process_static_image() finds and draws lane lines over test images

In [17]:
import os

def process_static_image(image):
    " Draw lanes on images"
    # filter out everything but the lanes
    img = lane_pass_filter(image)
    # convert to greyscale and apply blur
    gray = grayscale(img)
    blur_gray = gaussian_blur(gray,7)
    # edge dectection
    edges = canny(blur_gray,50,150)
    # create a mask for region of interest
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),((imshape[1]/2)-20,(imshape[0]/2)+50), ((imshape[1]/2)+20,(imshape[0]/2)+50), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(edges,vertices)
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    region_end_point = (imshape[0]/2)+60 # the topmost point to which lane lines will be drawn
    transformed_img = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap,region_end_point)
    weighted = weighted_img(transformed_img,image)
    
    return weighted

all_images = os.listdir("test_images/")
# loop at all test images
for image in all_images:
    img = mpimg.imread("test_images/"+image)
    # reset global variables to skip averaging lane co ordinates 
    reset_globals()
    # process the image
    img = process_static_image(img)
    # save image
    plt.imsave("test_images_output/"+image,img)
    
    
    


## Test on Videos

The functions process_image() and process_image_challenge() are nearly identical with just one variable changed to a different value. 

In [18]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [19]:
def process_image(image):
    
    # filter out everything but the lanes
    img = lane_pass_filter(image)
    # convert to greyscale and apply blur
    gray = grayscale(img)
    blur_gray = gaussian_blur(gray,7)
    # edge dectection
    edges = canny(blur_gray,50,150)
    # create a mask for region of interest
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),((imshape[1]/2)-20,(imshape[0]/2)+50), ((imshape[1]/2)+20,(imshape[0]/2)+50), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(edges,vertices)
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    region_end_point = (imshape[0]/2)+60 # top most point to which lane lines will be drawn
    transformed_img = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap,region_end_point)
    weighted = weighted_img(transformed_img,image)
    
    return weighted

def process_image_challenge(image):
    
    'Same as process_image() with a slightly greater value of region_end_point for the challenge clip '
    img = lane_pass_filter(image)
    gray = grayscale(img)
    blur_gray = gaussian_blur(gray,7)
    edges = canny(blur_gray,50,150)
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),((imshape[1]/2)-20,(imshape[0]/2)+80), ((imshape[1]/2)+20,(imshape[0]/2)+80), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(edges,vertices)
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    region_end_point = (imshape[0]/2)+80 # top most point to which lane lines will be drawn
    transformed_img = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap,region_end_point)
    weighted = weighted_img(transformed_img,image)
    return weighted 

Let's try the one with the solid white lane on the right first ...

In [20]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 221/222 [00:12<00:00, 22.83it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 14 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [22]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
# reset global variables between clips
reset_globals()
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:36<00:00, 18.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 38 s


In [23]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

It works !!

In [24]:
challenge_output = 'test_videos_output/challenge.mp4'
# reset global variables between clips
reset_globals()
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:26<00:00,  4.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 28.9 s


In [25]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))